In [ ]:
!pip install pyomo
!pip install haversine


In [ ]:

#Install Solvers
%%capture
import sys
import os

if True:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

In [ ]:
#!pip install pyomo
#!pip install haversine
import numpy as np
import pandas as pd
import os
import pyomo.environ as pyo
from haversine import haversine, Unit
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h1>Data Import and Model Preprocessing</h1>

In [ ]:
#airport data (already cleaned)
#change to desired path
ap = pd.read_csv(os.path.join("/","content", "drive", "MyDrive", "OPTI_Final", "DataC", "airports.csv"))
ap["co"] = ap.apply(lambda row: (row.lat, row.lon),axis=1)
ap.index = ap.index + 1
#county data (already cleaned)
#change to desired path
co = pd.read_csv(os.path.join("/","content", "drive", "MyDrive", "OPTI_Final", "DataC", "counties.csv"))
co["co"] = co.apply(lambda row: (row.lat, row.lon),axis=1)
co.index = co.index + 1

In [ ]:
#Sets
I = co.index.to_list()
J = ap.index.to_list()

#Params
D = {}
for i in I:
    county_loc = co.at[i, "co"]
    for j in J:
        airport_loc = ap.at[j, "co"]
        D[(i,j)] = haversine(county_loc, airport_loc, unit=Unit.MILES)

A = {i:co.at[i, "pop"] for i in I}

<h1>Model</h1>

In [ ]:
model = pyo.ConcreteModel()

<h2>Indexes</h2>

In [ ]:
#county index
model.I = pyo.Set(initialize=I)
#airport index
model.J = pyo.Set(initialize=J)

In [ ]:
print(I[0], I[-1])
print(J[0], J[-1])

1 1086
1 149


<h2>Parameters</h2>

In [ ]:
#distance between airports & counties
model.d = pyo.Param(model.I, model.J, initialize=D)
#population supply of the counties
model.a = pyo.Param(model.I, initialize=A)
#number of facilities to fly out of
model.p = pyo.Param(mutable=True, initialize=4)

<h2>Decision Variables</h2>

In [ ]:
model.X = pyo.Var(model.I, model.J, domain=pyo.Binary)
model.Y = pyo.Var(model.J, domain=pyo.Binary)

<h2>Objective</h2>

In [ ]:
def obj_rule(model):
    return(sum(sum(model.a[i]*model.d[(i,j)]*model.X[(i,j)] for j in model.J) for i in model.I))

model.obj = pyo.Objective(rule=obj_rule(model), sense=pyo.minimize)

<h2>Constraints</h2>

In [ ]:
model.facility_demand_con = pyo.ConstraintList()
for i in model.I:
    model.facility_demand_con.add(sum(model.X[i, j] for j in model.J) == 1)

In [ ]:
def num_facilities_constraint(model):
    return(sum(model.Y[j] for j in model.J) == model.p)
model.num_facilities_con = pyo.Constraint(rule=num_facilities_constraint(model))

In [ ]:
model.demand_supplied_con = pyo.ConstraintList()
for j in model.J:
    for i in model.I:
        model.demand_supplied_con.add(model.X[(i,j)] <= model.Y[j])

In [ ]:
constant_airport = int(ap[ap.Fac_Name == "BILL AND HILLARY CLINTON NATIONAL/ADAMS FIELD"].index[0])

In [ ]:
model.ark_airport_constraint = pyo.Constraint(expr = model.Y[constant_airport] == 1)

In [ ]:
solver = pyo.SolverFactory('cbc')
solver.solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17096282520.08644
  Upper bound: 17096282520.08644
  Number of objectives: 1
  Number of constraints: 161815
  Number of variables: 161962
  Number of binary variables: 161963
  Number of integer variables: 161963
  Number of nonzeros: 161814
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 26.36
  Wallclock time: 27.17
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is 

In [ ]:
model.Y.pprint()

Y : Size=149, Index=J
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary
      3 :     0 :   0.0 :     1 : False : False : Binary
      4 :     0 :   0.0 :     1 : False : False : Binary
      5 :     0 :   0.0 :     1 : False : False : Binary
      6 :     0 :   1.0 :     1 : False : False : Binary
      7 :     0 :   0.0 :     1 : False : False : Binary
      8 :     0 :   0.0 :     1 : False : False : Binary
      9 :     0 :   0.0 :     1 : False : False : Binary
     10 :     0 :   0.0 :     1 : False : False : Binary
     11 :     0 :   0.0 :     1 : False : False : Binary
     12 :     0 :   0.0 :     1 : False : False : Binary
     13 :     0 :   0.0 :     1 : False : False : Binary
     14 :     0 :   0.0 :     1 : False : False : Binary
     15 :     0 :   0.0 :     1 : False : False : Binary
     16 :     0 :   0.0 :     1 : False : False : Binary
     17 :

In [ ]:
for j in model.J:
  if pyo.value(model.Y[j]) == 1:
    print(f"Optimal airport: {j}, {ap.at[j, 'Fac_Name']}")

Optimal airport: 6, LAKELAND LINDER INTL
Optimal airport: 51, BILL AND HILLARY CLINTON NATIONAL/ADAMS FIELD
Optimal airport: 93, ANDERSON RGNL
Optimal airport: 132, EASTERWOOD FIELD


In [ ]:
avg = pyo.value(model.obj)/co["pop"].sum()
print(f"Average distance from airport: {round(avg,3)}")

Average distance from airport: 165.216
